In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
with open('train-en/left.txt') as fopen:
    left = fopen.read().split('\n')
    
with open('train-en/right.txt') as fopen:
    right = fopen.read().split('\n')

In [3]:
import malaya

/home/husein/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/husein/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [4]:
from malaya.text.rules import rules_normalizer, rules_compound_normalizer
from malaya.text.normalization import _is_number_regex
from malaya.text.function import replace_punct
from collections import defaultdict
import re
import random
import json

In [5]:
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

def strip_punct(word):
    left = []
    right = []
    i = 0
    while i < len(word):
        if word[i] in PUNCTUATION:
            left.append(word[i])
            i += 1
        else:
            break
    i = len(word) - 1
    while i > 0:
        if word[i] in PUNCTUATION:
            right.append(word[i])
            i -= 1
        else:
            break
    left = ''.join(left)
    right = ''.join(right[::-1])
    if len(right):
        word_ = word[:-len(right)]
    else:
        word_ = word
    word_ = word_[len(left):]
    return left, right, word_

In [6]:
def replace_words_punct(left_word, right_word):
    left_left, left_right, left_word = strip_punct(left_word)
    right_left, right_right, right_word = strip_punct(right_word)
    return f'{left_left}{right_word}{left_right}'

def random_replace_alignment(left, right, alignment, min_replace = 5, max_replace = 15):
    splitted_left = left.split()
    splitted_right = right.split()
    
    selected_alignment = []
    for s in alignment:
        l = s[0]
        r = s[1]
        try:
            if _is_number_regex(splitted_left[l].replace(',', '').replace('.', '')) or _is_number_regex(splitted_right[r].replace(',', '').replace('.', '')):
                continue
            elif splitted_left[l].isupper() or splitted_right[r].isupper():
                continue
            elif splitted_left[l].lower() == splitted_right[r].lower():
                continue
            elif splitted_left[r].lower() in ['the', 'a', 'an', 'it', 'is', 'are']:
                continue
            else:
                selected_alignment.append((l, r))
        except:
            pass
    
    try:
        count_replace = random.randint(min_replace, min(max_replace, len(selected_alignment)))
        selected = random.sample(selected_alignment, count_replace)
        for s in selected:
            splitted_left[s[0]] = replace_words_punct(splitted_left[s[0]], splitted_right[s[1]])

        return ' '.join(splitted_left), selected
    
    except:
        return ' '.join(splitted_left), []

In [7]:
eflomal = malaya.alignment.en_ms.eflomal(preprocessing_func=replace_punct)

In [8]:
model = malaya.language_detection.substring_rules()

In [9]:
tokenizer = malaya.tokenizer.Tokenizer()

In [10]:
l = ' '.join(tokenizer.tokenize(left[0]))
r = ' '.join(tokenizer.tokenize(right[0]))

In [11]:
%%time

alignment = eflomal.align([l], [r])['forward'][0]

CPU times: user 4.46 ms, sys: 44.2 ms, total: 48.6 ms
Wall time: 161 ms


In [12]:
r_ = random_replace_alignment(l, r, alignment)
r_

('Most ini them are remaja ataupun dalam lengkongan 20s .',
 [(1, 2), (7, 7), (5, 5), (4, 4), (6, 6)])

In [13]:
r_[1]

[(1, 2), (7, 7), (5, 5), (4, 4), (6, 6)]

In [14]:
r_[0]

'Most ini them are remaja ataupun dalam lengkongan 20s .'

In [15]:
predicted = model.predict(l.split())
for i in r_[1]:
    predicted[i[0]] = 'MS'
predicted

['EN', 'MS', 'EN', 'EN', 'MS', 'MS', 'MS', 'MS', 'NOT_LANG', 'NOT_LANG']

In [24]:
from tqdm import tqdm

def loop(rows):
    rows, _ = rows
    strings, labels = [], []
    for i in tqdm(range(len(rows))):
        try:
            left_, right_ = rows[i][0], rows[i][1]
            tokenized_l = tokenizer.tokenize(left_)
            tokenized_r = tokenizer.tokenize(right_)
            l = ' '.join(tokenized_l)
            r = ' '.join(tokenized_r)
            
            if len(tokenized_l) > 60 or len(tokenized_r) > 60:
                continue
                
            alignment = eflomal.align([l], [r])['forward'][0]
            r_ = random_replace_alignment(l, r, alignment)
            predicted = model.predict(l.split())
            for k in r_[1]:
                predicted[k[0]] = 'MS'
            strings.append(r_[0])
            labels.append(predicted)
        except Exception as e:
            print(e)
    
    return [[strings, labels]]

In [25]:
r = loop((list(zip(left[:10], right[:10])),0))

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.03it/s]


In [23]:
import mp

In [38]:
r = mp.multiprocessing(list(zip(left[200000:400000], right[200000:400000])), loop, cores = 4)

100%|████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [5:05:29<00:00,  2.73it/s]


In [39]:
strings, labels = [], []
for i in range(len(r)):
    print(i, len(r[i][0]))
    strings.extend(r[i][0])
    labels.extend(r[i][1])

0 42868
1 42709
2 42683
3 42770


In [40]:
strings[-1], labels[-1]

('Pada 2014 , beliau menjadi eksekutif penerbit kepada " Hungry Ghost Ritual " di the direction pelakon Hong Kong actor Nick Cheung .',
 ['MS',
  'NOT_LANG',
  'NOT_LANG',
  'MS',
  'MS',
  'MS',
  'MS',
  'MS',
  'NOT_LANG',
  'EN',
  'EN',
  'EN',
  'NOT_LANG',
  'MS',
  'EN',
  'EN',
  'MS',
  'EN',
  'EN',
  'EN',
  'EN',
  'EN',
  'NOT_LANG'])

In [41]:
with open('en-ms-substrings-v2.json', 'w') as fopen:
    json.dump({'strings': strings, 'labels': labels}, fopen)

In [42]:
with open('en-ms-substrings-v2.json') as fopen:
    data = json.load(fopen)

In [43]:
data['strings'][10]

'Jengka Teknik Menengah Sekolah Jengka Teknik Menengah Sekolah atau ringkasnya nama " SM Teknik Jengka " , is a Technical High Sekolah located in yang Central Bandar Pusat Jengka .'

In [44]:
data['labels'][10]

['MS',
 'MS',
 'MS',
 'MS',
 'MS',
 'MS',
 'MS',
 'MS',
 'MS',
 'MS',
 'MS',
 'NOT_LANG',
 'CAPITAL',
 'MS',
 'MS',
 'NOT_LANG',
 'NOT_LANG',
 'EN',
 'EN',
 'EN',
 'EN',
 'MS',
 'EN',
 'EN',
 'MS',
 'EN',
 'MS',
 'MS',
 'MS',
 'NOT_LANG']